1. Load data and model

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Check your setup.")


GPU is available!
Using GPU: NVIDIA GeForce RTX 4080 Laptop GPU


In [3]:
# Import required libraries
import os
from datasets import Dataset

# Function to read IOB files and split into paragraphs
def read_iob_file(file_path):
    """Read IOB file and convert to token-label samples"""
    examples = []
    words, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.split("\t")
                words.append(token)
                labels.append(label.strip())
            else:
                # End of a sample
                if words and labels:
                    examples.append({"tokens": words, "labels": labels})
                    words, labels = [], []
    # Add the last example if file doesn't end with a blank line
    if words and labels:
        examples.append({"tokens": words, "labels": labels})
    return examples



c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load datasets using the updated function
train_path = 'C:\\S24-25\\TxM\\dataset\\train.tsv'
val_path = 'C:\\S24-25\\TxM\\dataset\\val_gold.tsv'
test_path = 'C:\\S24-25\\TxM\\dataset\\test_gold.tsv'
train_data = read_iob_file(train_path)
val_data = read_iob_file(val_path)
test_data = read_iob_file(test_path)

print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}, Test samples: {len(test_data)}")


Train samples: 998, Val samples: 124, Test samples: 126


In [5]:
from transformers import AutoTokenizer
# Load model directly
# Use a pipeline as a high-level helper
from transformers import pipeline
model_checkpoint="google/electra-small-discriminator"
pipe = pipeline("fill-mask", model="google/electra-small-discriminator")
# Define label mappings
label_list = ["O", "B-ADR", "I-ADR", "B-DRU", "I-DRU", "B-DIS", "I-DIS", "B-SYM", "I-SYM"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def tokenize_and_align_labels(examples):
    tokenized_inputs = pipe.tokenizer(
        examples["tokens"], 
        is_split_into_words=True,
        padding="max_length",      # or True, depending on your preference
        truncation=True,          # enable truncation
        max_length=512,           # explicitly set maximum sequence length 
        return_overflowing_tokens=False,
    )
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore padding
            elif word_idx != previous_word_idx:
                aligned_labels.append(label_to_id[label[word_idx]])
            else:
                aligned_labels.append(label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize datasets
train_dataset = Dataset.from_list(train_data).map(tokenize_and_align_labels, batched=True)
test_dataset = Dataset.from_list(test_data).map(tokenize_and_align_labels, batched=True)
val_dataset = Dataset.from_list(val_data).map(tokenize_and_align_labels, batched=True)


c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HikaLipp\.cache\huggingface\hub\models--google--electra-small-discriminator. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at google/e

In [6]:
from transformers import AutoModelForTokenClassification


# Load the pre-trained BERT model

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. Train and evaluate with default parameters on the test set

In [10]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./electra_logs",
    logging_steps=10,
)


In [11]:
from transformers import Trainer
from evaluate import load
import torch

# Load metric for evaluation
metric = load("seqeval")

def compute_metrics(predictions):
    predictions, labels = predictions
    predictions = torch.argmax(torch.tensor(predictions), dim=2)
    
    # Convert predictions and labels to CPU and then to plain Python integers
    predictions = predictions.cpu().numpy()
    
    true_labels = [
        [id_to_label[label] for label in label_seq if label != -100] 
        for label_seq in labels
    ]
    true_predictions = [
        [id_to_label[int(p)] for (p, l) in zip(prediction, label_seq) if l != -100]  # Convert tensor to int
        for prediction, label_seq in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_predictions, references=true_labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=pipe.tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


  6%|▌         | 11/189 [00:01<00:22,  8.07it/s]

{'loss': 0.4572, 'grad_norm': 1.2515650987625122, 'learning_rate': 1.8941798941798943e-05, 'epoch': 0.16}


 11%|█         | 21/189 [00:02<00:20,  8.24it/s]

{'loss': 0.4637, 'grad_norm': 0.9536014199256897, 'learning_rate': 1.7883597883597884e-05, 'epoch': 0.32}


 16%|█▋        | 31/189 [00:03<00:19,  8.24it/s]

...

 99%|█████████▉| 188/189 [00:24<00:00,  8.37it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
100%|██████████| 189/189 [00:25<00:00,  8.37it/s]

{'eval_loss': 0.3368615508079529, 'eval_ADR': {'precision': 0.2594142259414226, 'recall': 0.3257443082311734, 'f1': 0.2888198757763975, 'number': 571}, 'eval_DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 52}, 'eval_DRU': {'precision': 0.8090909090909091, 'recall': 0.5913621262458472, 'f1': 0.6833013435700576, 'number': 301}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 48}, 'eval_overall_precision': 0.38847385272145146, 'eval_overall_recall': 0.37448559670781895, 'eval_overall_f1': 0.3813514929282347, 'eval_overall_accuracy': 0.9056971723081209, 'eval_runtime': 0.5631, 'eval_samples_per_second': 223.778, 'eval_steps_per_second': 14.208, 'epoch': 3.0}


100%|██████████| 189/189 [00:25<00:00,  7.45it/s]

{'train_runtime': 25.3804, 'train_samples_per_second': 117.965, 'train_steps_per_second': 7.447, 'train_loss': 0.38417816162109375, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=0.38417816162109375, metrics={'train_runtime': 25.3804, 'train_samples_per_second': 117.965, 'train_steps_per_second': 7.447, 'total_flos': 87493824681984.0, 'train_loss': 0.38417816162109375, 'epoch': 3.0})

In [12]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


 75%|███████▌  | 6/8 [00:00<00:00, 18.28it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 8/8 [00:00<00:00, 14.72it/s]

Evaluation Results: {'eval_loss': 0.3368615508079529, 'eval_ADR': {'precision': 0.2594142259414226, 'recall': 0.3257443082311734, 'f1': 0.2888198757763975, 'number': 571}, 'eval_DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 52}, 'eval_DRU': {'precision': 0.8090909090909091, 'recall': 0.5913621262458472, 'f1': 0.6833013435700576, 'number': 301}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 48}, 'eval_overall_precision': 0.38847385272145146, 'eval_overall_recall': 0.37448559670781895, 'eval_overall_f1': 0.3813514929282347, 'eval_overall_accuracy': 0.9056971723081209, 'eval_runtime': 0.7153, 'eval_samples_per_second': 176.139, 'eval_steps_per_second': 11.183, 'epoch': 3.0}


: 

3. Train and test on val set

4. HPO